In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q google-adk google-genai "uvicorn[standard]" requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.6/456.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<4

In [3]:
import os
import time
import uuid
import json
import subprocess
import requests
import logging
import warnings

In [4]:
from kaggle_secrets import UserSecretsClient

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

warnings.filterwarnings("ignore")


In [5]:
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ GOOGLE_API_KEY loaded from Kaggle secrets.")
except Exception as e:
    raise RuntimeError(
        f"❌ Could not load GOOGLE_API_KEY from Kaggle secrets. Details: {e}"
    )

✅ GOOGLE_API_KEY loaded from Kaggle secrets.


# 2. Configure basic logging (optional, but useful)

In [6]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)

print("✅ Logging configured.")

✅ Logging configured.


In [7]:
retry_cfg = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Base setup complete.")

✅ Base setup complete.


## Product Catalog Agent CODE (will be saved to /tmp/product_catalog_server.py)

In [8]:
product_catalog_server_code = r'''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_cfg = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

def get_product_info(product_name: str) -> str:
    """Get product information for a given product."""
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": "Dell XPS 15, $1,299, In Stock (45 units), 15.6 inch display, 16GB RAM, 512GB SSD",
        "macbook pro 14": "MacBook Pro 14, $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD",
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": "iPad Air, $599, In Stock (28 units), 10.9 inch display, 64GB",
        "lg ultrawide 34": "LG UltraWide 34 Monitor, $499, Out of Stock, Expected: Next week",
    }

    key = product_name.lower().strip()
    if key in product_catalog:
        return f"Product: {product_catalog[key]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"

product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_cfg),
    name="product_catalog_agent",
    description="External vendor product catalog that provides price, stock and basic specs.",
    instruction="""
    You are a product catalog specialist.
    For any product-related question, you MUST call get_product_info.
    Return clear, concise responses with price and stock status.
    """,
    tools=[get_product_info],
)

# A2A app
app = to_a2a(product_catalog_agent, port=8001)
'''

# -------------------------------------------------------------------
# Write server file
# -------------------------------------------------------------------
server_path = "/tmp/product_catalog_server.py"
with open(server_path, "w") as f:
    f.write(product_catalog_server_code)

print(f"📝 Product catalog server code written to: {server_path}")


📝 Product catalog server code written to: /tmp/product_catalog_server.py


## Start uvicorn server in background

In [9]:
env = os.environ.copy()

server_process = subprocess.Popen(
    [
        "uvicorn",
        "product_catalog_server:app",
        "--host",
        "127.0.0.1",
        "--port",
        "8001",
    ],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env=env,
)

print("🚀 Starting Product Catalog Agent (A2A) on http://127.0.0.1:8001 ...")

# -------------------------------------------------------------------
# Wait until agent card is reachable
# -------------------------------------------------------------------
agent_card_url = f"http://127.0.0.1:8001{AGENT_CARD_WELL_KNOWN_PATH}"
for i in range(30):
    try:
        r = requests.get(agent_card_url, timeout=1)
        if r.status_code == 200:
            print(f"✅ A2A Product Catalog Agent is up!")
            print(f"   Agent card: {agent_card_url}")
            break
    except requests.exceptions.RequestException:
        pass
    time.sleep(1)
else:
    print("⚠️ Could not confirm Product Catalog Agent is running. Check logs if needed.")

# Keep handle globally so we could kill it later if we want
product_catalog_server_process = server_process


🚀 Starting Product Catalog Agent (A2A) on http://127.0.0.1:8001 ...
✅ A2A Product Catalog Agent is up!
   Agent card: http://127.0.0.1:8001/.well-known/agent-card.json


In [10]:
try:
    resp = requests.get(agent_card_url, timeout=5)
    print("📋 Product Catalog Agent Card:")
    print(json.dumps(resp.json(), indent=2))
except Exception as e:
    print(f"❌ Failed to fetch agent card: {e}")


📋 Product Catalog Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor product catalog that provides price, stock and basic specs.",
  "name": "product_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor product catalog that provides price, stock and basic specs. \n    I am a product catalog specialist.\n    For any product-related question, I MUST call get_product_info.\n    Return clear, concise responses with price and stock status.\n    ",
      "id": "product_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Get product information for a given product.",
      "id": "product_catalog_agent-get_product_info",
      "name": "get_product_info",
      "tags": [
        "llm",
        "tools"
      ]
    }
  ],
  "supportsAuthenti

## 3) Create RemoteA2aAgent (Client Side)

In [11]:
remote_product = RemoteA2aAgent(
    name="product_catalog_agent",
    description="Remote product catalog (A2A) for price and availability.",
    agent_card=agent_card_url,
)

print("✅ RemoteA2aAgent for product catalog created.")


✅ RemoteA2aAgent for product catalog created.


## 4)Local Business Tools (Customer, Refund, Shipping) + Escalation Agent

In [12]:
# -------------------------------------------------------------------
# 1. Customer profile tool
# -------------------------------------------------------------------
def get_customer_profile(customer_id: str) -> dict:
    """
    Mock customer profile lookup.
    """
    profiles = {
        "211": {
            "customer_id": "211",
            "name": "Arin Jain",
            "tier": "GOLD",
            "lifetime_value_inr": 450000,
        },
        "101": {
            "customer_id": "101",
            "name": "Alice Singh",
            "tier": "SILVER",
            "lifetime_value_inr": 70000,
        },
        "501": {
            "customer_id": "501",
            "name": "Enterprise Corp.",
            "tier": "PLATINUM",
            "lifetime_value_inr": 2500000,
        },
    }

    if customer_id in profiles:
        return {"status": "found", "profile": profiles[customer_id]}
    else:
        return {
            "status": "not_found",
            "error": f"No profile for customer_id={customer_id}",
        }

In [13]:
# 2. Refund policy tool
# -------------------------------------------------------------------
def get_refund_policy(product_name: str, customer_tier: str) -> dict:
    """
    Mock refund policy logic.
    Limits are in INR.
    """
    tier = customer_tier.upper().strip()

    if tier == "SILVER":
        limit = 10000
    elif tier == "GOLD":
        limit = 20000
    elif tier == "PLATINUM":
        limit = 50000
    else:
        limit = 5000  # default / unknown

    return {
        "status": "ok",
        "product_name": product_name,
        "customer_tier": tier,
        "max_refund_inr": limit,
        "comment": f"Max refundable amount for tier {tier} is INR {limit}.",
    }

In [14]:
# 3. Shipping tool (simple, no ToolConfirmation)
# -------------------------------------------------------------------
def ship_order(order_id: str, units: int) -> dict:
    """
    Simple shipping tool.
    No confirmation, no pause/resume – always returns a result.
    """
    if units <= 0:
        return {
            "status": "error",
            "message": f"Invalid units: {units}. Units must be positive.",
        }

    priority = "standard"
    if units > 10:
        priority = "bulk"

    return {
        "status": "success",
        "order_id": order_id,
        "units": units,
        "priority": priority,
        "message": f"Shipment created for order {order_id} with {units} units ({priority} shipping).",
    }

In [15]:
# 4. Escalation agent (Tier 2 support)
# -------------------------------------------------------------------
escalation_agent = LlmAgent(
    name="escalation_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_cfg),
    description="Tier-2 escalation agent that summarizes complex cases for human review.",
    instruction="""
    You are Tier-2 enterprise support.
    Your job is to generate a short, structured summary for a human agent.

    Always include:
    - Customer ID and tier (if known)
    - Product
    - Amount or units involved
    - Why this case was escalated (e.g. refund over limit, unusual pattern)
    - Clear next steps for the human agent

    Keep it under 8 bullet points.
    """,
)

print("✅ Business tools + escalation_agent ready.")

✅ Business tools + escalation_agent ready.


## 5) Enterprise Support Agent (Main Brain)

In [16]:
support_agent = LlmAgent(
    name="enterprise_support_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_cfg),
    description="Enterprise support assistant handling product info, refunds, and shipping.",
    instruction="""
    You are an enterprise support assistant.

    You have these capabilities:
    - Remote product catalog via product_catalog_agent (A2A sub-agent)
    - Local tools: get_customer_profile, get_refund_policy, ship_order
    - Escalation via escalation_agent

    HARD RULES (YOU MUST FOLLOW THEM):
    1. If user asks about a product (price/stock/specs), call product_catalog_agent.
    2. For refund:
       - Extract: product, customer_id, refund_amount_inr from the message if present.
       - Call get_customer_profile(customer_id).
       - Call get_refund_policy(product_name, customer_tier).
       - If requested refund_amount_inr > max_refund_inr:
           -> Call escalation_agent with a brief summary.
           -> Tell the user that the case has been escalated and cannot be auto-approved.
       - Otherwise:
           -> Approve the refund and clearly state amount and policy.
    3. For shipping:
       - Extract order_id and units.
       - Call ship_order(order_id, units) directly.
       - If status=success, confirm shipment details.
       - DO NOT use any confirmation/pause mechanism. Always return a final decision.
    4. If the user message already contains all required information
       (customer id, product, amount, units, order id),
       DO NOT ask follow-up questions. Just proceed with tools and respond.
    5. Responses must be CLEAR and STRUCTURED, e.g.:

       **Result**
       - ...

       **Details**
       - ...

    Never pretend to have done actions outside tools.
    """,
    tools=[
        get_customer_profile,
        get_refund_policy,
        ship_order,
        AgentTool(escalation_agent),
    ],
    sub_agents=[remote_product],
)

print("✅ enterprise_support_agent created.")


✅ enterprise_support_agent created.


## 6) Runner + Helper ask() Function

In [17]:
session_service = InMemorySessionService()

async def ask(user_query: str):
    """
    Convenience helper to talk to enterprise_support_agent in Kaggle.
    """
    app_name = "enterprise_support_app"
    user_id = "demo_user"
    session_id = f"sess_{uuid.uuid4().hex[:6]}"

    # Create session (IMPORTANT: use keyword arguments)
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
    )

    runner = Runner(
        agent=support_agent,
        app_name=app_name,
        session_service=session_service,
    )

    content = types.Content(parts=[types.Part(text=user_query)])

    print(f"🧑: {user_query}")
    print("🤖:")

    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=content,
    ):
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    print(part.text)
    print("-" * 60)


## 7) Test Calls

In [18]:
await ask("Tell me about the Macbook Pro 14")

🧑: Tell me about the Macbook Pro 14
🤖:


The MacBook Pro 14 is available for $1,999 and is currently in stock with 22 units available. It features an M3 Pro chip, 18GB of RAM, and a 512GB SSD.
------------------------------------------------------------


In [19]:
await ask("Refund Macbook Pro 14 for customer 211 amount 18000 INR")

🧑: Refund Macbook Pro 14 for customer 211 amount 18000 INR
🤖:


**Refund Approved**

**Amount:** 18000 INR
**Policy:** Your refund for the Macbook Pro 14 has been approved according to the GOLD tier policy, which allows a maximum refund of 20000 INR.
**Order Confirmation:** Shipment created for order 211 with 18000 units (bulk shipping).
------------------------------------------------------------


In [20]:
await ask("Refund Macbook Pro 14 for customer 210 amount 18000 INR")

🧑: Refund Macbook Pro 14 for customer 210 amount 18000 INR
🤖:


I am sorry, but I could not find a profile for customer ID 210. Please check the customer ID and try again.
------------------------------------------------------------


In [21]:
await ask("Ship 12 units of order 9921")

🧑: Ship 12 units of order 9921
🤖:


**Result**
- Order 9921 has been shipped successfully.

**Details**
- Units shipped: 12
- Shipping priority: bulk
------------------------------------------------------------


In [22]:
await ask("Ship 5 units of order 9921")

🧑: Ship 5 units of order 9921
🤖:


**Result**
Order 9921 has been shipped with 5 units using standard shipping.
------------------------------------------------------------
